In [ ]:
import pandas as pd

from loadData import loadData
from extract_fiducials import extract_fiducials
from extract_features import extract_features
from preprocess import preprocess
from artifacts import artifacts
import os

from sklearn.preprocessing import MinMaxScaler

import numpy as np

# Load Data

In [ ]:
ROOT_PATH = 'C:/Users/user/Desktop/AI-project/train'

In [ ]:
ppgs_128, speaks_128, annotations_128, ppgs_250, speaks_250, annotations_250 = loadData(ROOT_PATH)

# Pre-processing

In [ ]:
for i in range(len(ppgs_128)):
    ppgs_128[i] = preprocess(ppgs_128[i],128)

In [ ]:
for i in range(len(ppgs_250)):
    ppgs_250[i] = preprocess(ppgs_250[i],250)

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots()
axs.set_title("Patient 001_128")
fs=128
t = np.arange(0, ppgs_128[0].shape[0]/fs, 1/fs)
axs.plot(t[0:200*128], ppgs_128[0][0:200*128], color='C0')
axs.plot((speaks_128[0][0:300])/128, ppgs_128[0][speaks_128[0][0:300]],'*', color='r')
axs.set_xlabel("Time [s]")
axs.set_ylabel("Amplitude")
plt.show()

## Artifacts removal

In [ ]:
n_speaks_128=list()
n_annotations_128=list()

for i in range(len(speaks_128)):
    speaks, annotations = artifacts(speaks_128[i],ppgs_128[i],annotations_128[i])
    n_speaks_128.append(speaks)
    n_annotations_128.append(annotations)


In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots()
axs.set_title("Patient 001_128")
fs=128
t = np.arange(0, ppgs_128[0].shape[0]/fs, 1/fs)
axs.plot(t[0:200*128], ppgs_128[0][0:200*128], color='C0')
axs.plot((n_speaks_128[0][0:200])/128, ppgs_128[0][n_speaks_128[0][0:200]],'*', color='r')
axs.set_xlabel("Time [s]")
axs.set_ylabel("Amplitude")
plt.show()



In [ ]:
n_speaks_250=list()
n_annotations_250=list()

for i in range(len(speaks_250)):
    speaks, annotations = artifacts(speaks_250[i],ppgs_250[i],annotations_250[i])
    n_speaks_250.append(speaks)
    n_annotations_250.append(annotations)
    

In [ ]:
n_speaks_128[1]

# Extract fiducials

In [ ]:

fiducials_128 = list()

for i in range(len(ppgs_128)):
   fiducials_128.append(extract_fiducials(ppgs_128[i], n_speaks_128[i],128))


In [ ]:

fiducials_250 = list()

for i in range(len(ppgs_250)):
   fiducials_250.append(extract_fiducials(ppgs_250[i], n_speaks_250[i],250))


# Write fiducials to CSV
Because the process of extracting fiducials is time consuming, we write them on files

In [ ]:

i = 0
for fiducial in fiducials_128:
    file_name = f"C:/Users/user/Desktop/AI-project/fiducials/128/fiducial_{i}_128.csv"
    fiducial.to_csv(file_name, sep=',', index=False, encoding='utf-8')
    i = i + 1


In [ ]:

i = 0
for fiducial in fiducials_250:
    file_name = f"C:/Users/user/Desktop/AI-project/fiducials/250/fiducial_{i}_250.csv"
    fiducial.to_csv(file_name, sep=',', index=False, encoding='utf-8')
    i = i + 1


# Extract features

In [43]:

folder_path = u'C:/Users/user/Desktop/AI-project/fiducials/128' 

# List all files in the folder
file_names = [file for file in os.listdir(folder_path) if file.endswith('_128.csv')]

fiducials_128_copy = []

i = 0
# Loop through each file and concatenate the data
for i in range(len(file_names)):
    file_path = f"C:/Users/user/Desktop/AI-project/fiducials/128/fiducial_{i}_128.csv"
    fiducial = pd.read_csv(file_path)
    fiducials_128_copy.append(fiducial)

In [40]:

folder_path = u'C:/Users/user/Desktop/AI-project/fiducials/250' 

# List all files in the folder
file_names = [file for file in os.listdir(folder_path) if file.endswith('_250.csv')]

fiducials_250_copy = []

i = 0
# Loop through each file and concatenate the data
for i in range(len(file_names)):
    file_path = f"C:/Users/user/Desktop/AI-project/fiducials/128/fiducial_{i}_250.csv"
    fiducial = pd.read_csv(file_path)
    fiducials_250_copy.append(fiducial)

range(0, 62)

In [50]:
features_128 = []

for i in range(len(fiducials_128_copy)):
    features_128.append(extract_features(fiducials_128_copy[i], ppgs_128[i], 128))

TypeError: 'NoneType' object is not callable

In [ ]:
features_250 = []

for i in range(len(fiducials_250_copy)):
    features_250.append(extract_features(fiducials_250_copy[i], ppgs_250[i], 250))


# Add annotation to peaks

In [ ]:
for i in range(len(features_128)):
    features_128[i]['annotation'] = n_annotations_128[i]
for i in range(len(features_250)):
    features_250[i]['annotation'] = n_annotations_250[i]

# Remove rows with null values

In [ ]:
for i in range(len(features_128)):
    # Find rows with at least one NaN value using np.isnan()
    rows_with_nan = features_128[i][features_128[i].applymap(lambda x: np.isnan(x) if isinstance(x, (float, np.float64)) else False).any(axis=1)]

    # Remove rows with NaN values
    features_128[i] = features_128[i].drop(rows_with_nan.index)

In [ ]:
for i in range(len(features_250)):
    # Find rows with at least one NaN value using np.isnan()
    rows_with_nan = features_250[i][features_250[i].applymap(lambda x: np.isnan(x) if isinstance(x, (float, np.float64)) else False).any(axis=1)]

    # Remove rows with NaN values
    features_250[i] = features_250[i].drop(rows_with_nan.index)

# Remove rows with TP = 0 --> ST = inf

In [ ]:
for i in range(len(features_128)):
    features_128[i] = features_128[i][features_128[i]['TP'] != 0]

In [ ]:
for i in range(len(features_250)):
    features_250[i] = features_250[i][features_250[i]['TP'] != 0]

# Normalization

In [ ]:
import pandas as pd
for i in range(len(features_128)):
    
   normalized_df = features_128[i].iloc[:, :-1].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
   features_128[i] = pd.concat([normalized_df, features_128[i].iloc[:, -1]], axis=1)

In [ ]:
for i in range(len(features_250)):
   normalized_df = features_250[i].iloc[:, :-1].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
   features_250[i] = pd.concat([normalized_df, features_250[i].iloc[:, -1]], axis=1)

# Delete eventual NaN caused by Normalization

In [ ]:
for i in range(len(features_128)):
    # Find rows with at least one NaN value using np.isnan()
    rows_with_nan = features_128[i][features_128[i].applymap(lambda x: np.isnan(x) if isinstance(x, (float, np.float64)) else False).any(axis=1)]

    # Remove rows with NaN values
    features_128[i] = features_128[i].drop(rows_with_nan.index)

In [ ]:
for i in range(len(features_250)):
    # Find rows with at least one NaN value using np.isnan()
    rows_with_nan = features_250[i][features_250[i].applymap(lambda x: np.isnan(x) if isinstance(x, (float, np.float64)) else False).any(axis=1)]

    # Remove rows with NaN values
    features_250[i] = features_250[i].drop(rows_with_nan.index)

In [ ]:

for i in range(len(features_128)):
    features_128[i] = features_128[i][features_128[i]['TP'] != 0]
for i in range(len(features_250)):
    features_250[i] = features_250[i][features_250[i]['TP'] != 0]

In [ ]:
features_250[1]

# Write features to csv

In [ ]:
i = 0
for feature in features_128:
    file_name = f"C:/Users/cricr/PycharmProjects/AI-project/features/128/features_{i}_128.csv"
    feature.to_csv(file_name, sep=',', index=False, encoding='utf-8')
    i = i + 1

In [ ]:
i = 0
for feature in features_250:
    file_name = f"C:/Users/cricr/PycharmProjects/AI-project/features/250/features_{i}_250.csv"
    feature.to_csv(file_name, sep=',', index=False, encoding='utf-8')
    i = i + 1